In [87]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [88]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family='NanumGothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="NanumGothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))

warnings.filterwarnings("ignore")

In [89]:
import surprise
print(surprise.__version__)

1.1.1


In [90]:
from surprise import Reader
from surprise.model_selection import train_test_split

df = pd.read_csv('3kki_v2.csv')
df.head()

,memberno,productno,rating
0,72,21,3.0
1,47,15,3.0
2,77,23,1.0
3,27,8,2.0
4,75,1,1.0


In [91]:
reader = Reader(rating_scale=(1, 5))

In [92]:
from surprise import Dataset
data = Dataset.load_from_df(df, reader)

In [93]:
data

In [94]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

In [95]:
from surprise import SVD

algo = SVD()
algo.fit(trainset)

In [96]:
# 사용자 아이디(uid), 아이템 아이디(iid)는 문자열로 입력
uid = str(50)
iid = str(9)
# 사용자 아이디(uid), 아이템 아이디(iid)는 문자열로 입력
# 추천 예측 평점 (.predict)
pred = algo.predict(uid, iid)
pred

Prediction(uid='50', iid='9', r_ui=None, est=3.5294117647058822, details={'was_impossible': False})

In [97]:
# 추천 예측 평점 (.test)
predictions = algo.test( testset )

print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')

predictions[:5]

prediction type : <class 'list'>  size: 125
prediction 결과의 최초 5개 추출


[Prediction(uid=44, iid=16, r_ui=5.0, est=3.6614166960964085, details={'was_impossible': False}),
 Prediction(uid=71, iid=13, r_ui=3.0, est=3.7996576321404785, details={'was_impossible': False}),
 Prediction(uid=78, iid=7, r_ui=4.0, est=3.3507957955745145, details={'was_impossible': False}),
 Prediction(uid=19, iid=1, r_ui=2.0, est=3.700450584350457, details={'was_impossible': False}),
 Prediction(uid=37, iid=15, r_ui=3.0, est=3.118441095751617, details={'was_impossible': False})]

In [98]:
# 속성 확인
[ (pred.uid, pred.iid, pred.est, pred.details) for pred in predictions[:3] ]

[(44, 16, 3.6614166960964085, {'was_impossible': False}),
 (71, 13, 3.7996576321404785, {'was_impossible': False}),
 (78, 7, 3.3507957955745145, {'was_impossible': False})]

In [99]:
from surprise import accuracy

# 성능 평가
accuracy.rmse(predictions)

RMSE: 1.1663


1.1663392031757978

In [100]:
# index와 header를 제거한 ratings_noh.csv 파일 생성
ratings = pd.read_csv('3kki_v2.csv')
ratings.to_csv('3kki_surprise_noh.csv', index=False, header=False)

In [101]:
from surprise import Reader
from surprise import Dataset
# Reader 객체 생성
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0.5, 5))

data = Dataset.load_from_file('3kki_surprise_noh.csv', reader=reader)
data 

In [102]:
# surprise의 train_test_split() 사용
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

# SVD를 이용한 잠재 요인 협업 필터링 (잠재 요인 크기 = 50)
algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)

# 추천 예측 평점 (.test)
predictions = algo.test( testset )

# 성능 평가
accuracy.rmse(predictions)

RMSE: 1.1584


1.1584229241262354

In [103]:
# load_from_df()을 이용하여 데이터 프레임을 surprise 데이터 셋으로 로딩 가능하다.
# 주의할 점은 컬럼 순서가 반드시 사용자 아이디, 아이템 아이디, 평점 순이어야 한다.
# 데이터 불러오기 (데이터 프레임)
ratings = pd.read_csv('3kki_v2.csv') 

# Reader 객체 생성
reader = Reader(rating_scale=(0.5, 5.0))

# 사용자 아이디, 아이템 아이디, 평점 순서 (원래는 timestamp도 있으나 제외)
data = Dataset.load_from_df(ratings[['memberno','productno', 'rating']], reader)

# surprise의 train_test_split() 사용
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# SVD를 이용한 잠재 요인 협업 필터링 (잠재 요인 크기 = 50)
algo = SVD(n_factors=5, random_state=0)
algo.fit(trainset)

# 추천 예측 평점 (.test)
predictions = algo.test( testset )

# 성능 평가
accuracy.rmse(predictions)

RMSE: 1.1509


1.15087337654385

### 교차 검증

In [104]:
import time
from surprise.model_selection import cross_validate 


# 데이터 불러오기 (데이터 프레임)
ratings = pd.read_csv('3kki_v2.csv') 

# Reader 객체 생성
reader = Reader(rating_scale=(0.5, 5.0))

# 사용자 아이디, 아이템 아이디, 평점 순서 (원래는 timestamp도 있으나 제외)
data = Dataset.load_from_df(ratings[['memberno','productno', 'rating']], reader)

# SVD를 이용한 잠재 요인 협업 필터링
algo = SVD(random_state=0)

start = time.time()
# 교차 검증 수행
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# surprise.model_selection의 cross_validate()으로 교차 검증이 가능하다.
end = time.time()
print('Runtime: {0:.2f} 초'.format(end-start))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1764  1.2393  1.1801  1.1221  1.0496  1.1535  0.0639  
MAE (testset)     0.9637  1.0078  0.9878  0.9432  0.8594  0.9524  0.0513  
Fit time          0.09    0.06    0.03    0.04    0.03    0.05    0.02    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Runtime: 0.26 초


## 하이퍼 파라미터

In [105]:
from surprise.model_selection import GridSearchCV

# n_epochs: SGD 수행 시 반복 횟수, n_factors: 잠재 요인 크기
param_grid = {
    'n_epochs': [10, 30, 40], 
    'n_factors': [30, 70, 160]
}

# GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3) # algo가 아닌 SVD 입력하였다.

start = time.time()
gs.fit(data)
end = time.time()
print('Runtime: {0:.2f} 초'.format(end-start))

# 최적 하이퍼 파라미터 및 그 때의 최고 성능
print(gs.best_params['rmse'])
print(gs.best_score['rmse'])

Runtime: 1.26 초
{'n_epochs': 10, 'n_factors': 30}
1.1359194203648713


In [106]:
# DatasetAutoFolds() 객체를 생성 후 build_full_trainset() 메서드를 호출하면 전체 데이터를 학습 데이터로 만들 수 있음
from surprise.dataset import DatasetAutoFolds

# Reader 객체 생성
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0.5, 5))

# DatasetAutoFolds 객체 생성
data_folds = DatasetAutoFolds(ratings_file='3kki_surprise_noh.csv', reader=reader)

# 전체 데이터를 train으로 지정
trainset = data_folds.build_full_trainset()
trainset

In [107]:
# SVD를 이용한 잠재 요인 협업 필터링 (잠재 요인 크기 = 50)
model = SVD(n_epochs=20, n_factors=50, random_state=0)
model.fit(trainset)  # 훈련

# 사용자 아이디, 아이템 아이디 문자열로 입력
uid = str(50)
iid = str(28)

# 추천 예측 평점 (.predict)
pred = model.predict(uid, iid, verbose=True) 

user: 50         item: 28         r_ui = None   est = 3.56   {'was_impossible': False}


In [108]:
# 상품 대한 상세 속성 정보 DataFrame로딩
products = pd.read_csv('3kki_product.csv')

# 아직 보지 않은 상품 리스트 함수
def get_unseen_surprise(ratings, products, memberno):
     # 특정 userId가 평점을 매긴 모든 상품 리스트
    seen_products = ratings[ratings['memberno']== memberno]['productno'].tolist()
    
    # 모든 상품명을 list 객체로 만듬. 
    total_products = products['productno'].tolist()
      
    # 한줄 for + if문으로 안 본 상품 리스트 생성
    unorder_product = [ product for product in total_products if product not in seen_products]
    
    # 일부 정보 출력
    total_product_cnt = len(total_products)
    seen_cnt = len(seen_products)
    unseen_cnt = len(unorder_product)
    
    print(f"전체 상품 수: {total_product_cnt}, 평점 매긴 상품 수: {seen_cnt}, 추천 대상 상품 수: {unseen_cnt}")
    
    return unorder_product

In [115]:
unseen_products = get_unseen_surprise(ratings, products, 3)

전체 상품 수: 25, 평점 매긴 상품 수: 7, 추천 대상 상품 수: 18


In [123]:
def recomm_product_by_surprise(algo, userId, unseen_products, top_n=10):
    
    # 아직 보지 않은 영화의 예측 평점: prediction 객체 생성
    predictions = []    
    for productno in unseen_products:
        predictions.append(algo.predict(str(userId), str(productno)))
    
    # 리스트 내의 prediction 객체의 est를 기준으로 내림차순 정렬
    def sortkey_est(pred):
        return pred.est

    predictions.sort(key=sortkey_est, reverse=True) # key에 리스트 내 객체의 정렬 기준을 입력
    
    # 상위 top_n개의 prediction 객체
    top_predictions = predictions[:top_n]
    
    # 영화 아이디, 제목, 예측 평점 출력
    print(f"Top-{top_n} 추천 영화 리스트")
    
    for pred in top_predictions:
        product_id = int(pred.iid)
        product_title = products[products["productno"] == product_id]["name"].tolist()
        product_rating = pred.est
        
        print(f"{product_title}: {product_rating:.2f}")

In [124]:
recomm_product_by_surprise(model, 3, unseen_products, top_n=25)

Top-25 추천 영화 리스트
['아워홈 참나무향 닭가슴살']: 3.90
['잇메이트 저염 훈제닭가슴살']: 3.90
['성수동 스무디 오트밀']: 3.87
['굽네 훈제 닭가슴살 오리지널']: 3.86
['유산균슬림다이어트']: 3.83
['애터미 밸런스라이프']: 3.82
['리뉴슬림 블루']: 3.81
['내가 몸짱이 될 샐러드 닭가슴살 & 메추리알 샐러드']: 3.80
['후디스 그릭요거트 무설탕 저지방']: 3.69
['잇메이트 닭가슴살소세지볶음밥 마늘맛']: 3.67
['HOPE 발효효소 밸런스밀 블랙']: 3.64
['맛있닭 닭가슴살 소시지 훈제맛']: 3.61
['단백질공화국 다크초콜릿']: 3.58
['수비드닭가슴살빅샐러드']: 3.50
['글램디 5킬로칼로리 워터젤리 망고맛']: 3.50
['아임닭 23 단백질 닭가슴살 샐러드']: 3.37
['더 담아 꽉~채운 계란야채곤약볶음밥&토마토소스닭가슴살슬라이스']: 3.28
['린 컴플리트']: 3.21


In [125]:
import pickle # 파이썬 자료형 저장 지원
import joblib # 병렬 프로그래밍, 모델 저장, 로딩

joblib.dump(model, './ml-latest-small.pkl') 

['./ml-latest-small.pkl']